# T2.0 INSTANCE SEGMENTATION

In this script, we will segment individual instances of floors, ceilings, walls, columns and doors. 
To this end, we modify oneformer3D repo (https://github.com/oneformer3d/oneformer3d)

## Steps to get OneFormer3D running

First, we install prequisites: 

- We install windows WSL in terminal to get access to Windows Subsystem for Linux (wsl --install)
- WSL2 is our default, with an Ubuntu installation
- We install docker (https://docs.docker.com/desktop/install/windows-install/). this serves a similar function as Anaconda but has entire containers
- We install docker extension in VS Code
- we intalled WSL extension in VS Code

Next, we will open the environment:
- Open VS CODE, and connect to WSL as you would to a Server (bottom left corner)
- Clone repo in WSL home directory (https://github.com/oneformer3d/oneformer3d.git)
- select python version 3.10 (bottom right corner)
- install docker extension (because you now have a new python version)
- Create docker file (don't do this is a docker is already present)
    - press F1 
    - add docker files to workspace 
    - select python: General as application platform
    - select main python file (tools/test or train.py)
    - don't create docker compose files

- Build docker image (based on dockerfile) - FAILS FOR WSL and WINDOWS
    - F1: Docker images: build image -> this works
    - we have a problem with the MinkowskiEngine building. maybe we have a incompatible GPU.
     RUN TORCH_CUDA_ARCH_LIST="6.1 7.0 8.6" \
    pip install git+https://github.com/NVIDIA/MinkowskiEngine.git@02fc608bea4c0549b0a7b00ca1bf15dee4a0b228 -v --no-deps \
    --install-option="--blas=openblas" \
    --install-option="--force_cuda" 


- Run docker container
    -F1
    - Run docker file
    - ensure docker engine is running 

NEXT, TRY TO INSTALL ON GPU SERVER

## MASK3D

we will set up Mask 3D on GPU server

 - Connect to GPU server (mbassier@10.130.0.45)
 - install environment based on repo instructions
 - torch-scatter gave a bug
 - put some data on the K drive
 - create symbolic link to the data from the repo
 - conda activate mask3d_cuda113  

 for preprocessing
 
 - data_dir='/srvgentjkd98p2/K/Projects/2018-12 Project PHD Maarten Bassier/5.Data/2019-04 Benchmark S3DIS Stanford/Stanford3dDataset_v1.2/'
 - save_dir='/geomatics/gpuserver-0/mbassier/2024-03 S3DIS Mask3D/'
 - python -m datasets.preprocessing.s3dis_preprocessing preprocess

 
 

In [ ]:

cfg_file='/home/mbassier/code/Open3D-ML/ml3d/configs/randlanet_elia2.yml'

dataset_path='/geomatics/gpuserver-0/mbassier/2023-03 Elia Substation chuncks/'



## LIBRARIES

In [1]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy

import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

#import utils
from context import utils
import utils as utl
import utils.t1_utils as t1


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [4]:
pcdPathLas=Path(os.getcwd()).parents[1]/'data'/'steel.las'
pcdPathLaz=Path(os.getcwd()).parents[1]/'data'/'steel.laz'

### IMPORT PCD

In [3]:
laz  = laspy.read(pcdPathLaz)
pcd=gmu.las_to_pcd(laz)
pcdNode=PointCloudNode(resource=pcd)

In [22]:
print(len(laz.xyz))
o3d.visualization.draw_geometries([pcd])

4073605


### import entire folder of PCD's

In [8]:
files=ut.get_list_of_files(Path(os.getcwd()).parents[1]/'data')

# note that this will start to show memory issues if you load all pcd's at once
# reading 5 million points takes 30s
pcdNodes=[]
for f in files:
    if f.endswith('laz'):
        laz  = laspy.read(pcdPathLaz)
        pcdNodes.append(PointCloudNode(resource=gmu.las_to_pcd(laz), name=ut.get_filename(f)))
        print(f'imported {f}')

print(f'Imported {len(pcdNodes)} point clouds.')

imported c:/Users/Maarten/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/E003.laz
imported c:/Users/Maarten/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/steel.laz
Imported 2 point clouds.
